# Heterogeneous robots achieve homeostasis in a dynamic environment through continuous adaptation and interaction

Include relevant libraries

In [ ]:
using Plots
using Statistics

Setup experimental and I/O constants

In [ ]:
DATA_PATH =             "../th"
VIDEO_FILE =            "video.mp4"

ROBOTS_COUNT =          25
TARGET_REPLICAS_COUNT = 100
EPOCHS_COUNT =          20_000
EPOCH_DURATION =        200

START_EPOCH =           0
CLUSTERING_RANGE =      0.3;

Count the number of completed replicas

In [ ]:
ACTUAL_REPLICAS_COUNT = count(
    f -> startswith(f, "performance_r"),
    readdir(DATA_PATH)
) / ROBOTS_COUNT
ACTUAL_REPLICAS_COUNT = round(Int64, ACTUAL_REPLICAS_COUNT)

"Run replicas: $ACTUAL_REPLICAS_COUNT"

Count the number of completed epochs

In [ ]:
COMPLETED_EPOCHS = round(Int32, filesize("$DATA_PATH/clusters.dat") / 4 / ROBOTS_COUNT / EPOCH_DURATION)
COMPLETED_EPOCHS = min(COMPLETED_EPOCHS, EPOCHS_COUNT)

In [ ]:
# TODO temporary
ACTUAL_REPLICAS_COUNT = 1;

## Performance based measures

Create a vector where to store all the performance of the robots

In [ ]:
P = Matrix{Float32}(undef, ACTUAL_REPLICAS_COUNT * ROBOTS_COUNT, COMPLETED_EPOCHS);

Load the data into the matrix

In [ ]:
data = Vector{Float32}(undef, COMPLETED_EPOCHS)

for i in 1:ACTUAL_REPLICAS_COUNT
    for j in 1:ROBOTS_COUNT
        seed = j * TARGET_REPLICAS_COUNT + i - 1

        file = open("$DATA_PATH/performance_r$seed.dat", "r", lock=false)

        read!(file, data)
        
        P[(i - 1) * ROBOTS_COUNT + j, :] = data

        close(file)
    end
end

### Plot the average robot performance across the epochs

In [ ]:
plot(
    mean(P, dims=1)[1:COMPLETED_EPOCHS],
    xlabel="Epochs", ylabel="Performance [AVG]",
    legend=false
)
plot!([EPOCHS_COUNT / 2], linestyle=:dash, seriestype=:vline, xlims=(0, COMPLETED_EPOCHS))

## Cluster based measures

In [ ]:
file = open("$DATA_PATH/clusters.dat", "r", lock=false)

C = Matrix{Int32}(undef, ACTUAL_REPLICAS_COUNT * COMPLETED_EPOCHS, EPOCH_DURATION)
s = Vector{Int32}(undef, ROBOTS_COUNT)

for i in 1:ACTUAL_REPLICAS_COUNT * COMPLETED_EPOCHS
    for j in 1:EPOCH_DURATION
        read!(file, s)

        C[i, j] = length(Set(s))    
    end
end

close(file)

### Clusters count

In [ ]:
plot(
    mean(C, dims=2)[1:COMPLETED_EPOCHS],
    xlabel="Epochs", ylabel="Clusters count [AVG]",
    legend=false
)
plot!([EPOCHS_COUNT / 2], linestyle=:dash, seriestype=:vline, xlims=(0, COMPLETED_EPOCHS))